<a href="https://colab.research.google.com/github/cludiaong/Front-end-additional/blob/master/copy%20of%20NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q snscrape==0.3.4

In [ ]:
# Import library
import datetime as dt
import re
import string
import snscrape.modules.twitter as sntwitter
import tweepy
import pandas as pd
import ast
pd.options.mode.chained_assignment = None
import numpy as np
seed = 0
np.random.seed(seed)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style = 'whitegrid')



from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from wordcloud import WordCloud

from multiprocessing import Pool
from functools import partial
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, LSTM
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#menentukan maximal tweets yang akan diambil
maxTweets = 5000
tweets_list2 = []

#mencari data tweets sesuai keyword
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('vaksin covid since:2020-03-31 until:2020-12-31 lang:id').get_items()):
    if i>maxTweets:
      break
    tweets_list2.append([tweet.date,tweet.content, tweet.username])

In [ ]:
tweets_df2 = pd.DataFrame(tweets_list2, columns=['Datetime','Text', 'Username'])

tweets_df2

In [ ]:
tweets_df2.to_csv('drive/My Drive/5000dataset.csv', sep=',', index=False)


In [ ]:
tweets = pd.read_csv('drive/My Drive/5000dataset.csv')
tweets

In [ ]:
!wget https://github.com/evanmartua34/Twitter-COVID19-Indonesia-Sentiment-Analysis---Lexicon-Based/raw/master/cleaning_source/update_combined_slang_words.txt
!ls

In [ ]:
# Download data untuk tokenisasi dan penghapusan stop words

slangJson = {}  #mendefinsikan dictionary kosong yang akan dipopulate
                #dengan data kata-kata slang dari file update_combined_slang_words.txt

# parse file menjadi sebuah python Dictionary
with open('update_combined_slang_words.txt', 'r') as slangFile:
  slangJson = ast.literal_eval(slangFile.read())

print(slangJson)
print(slangJson['@'])

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

contoh = "org2 pd gak percaya covid"
contoh = word_tokenize(contoh)

for i in range(len(contoh)):
  if contoh[i] in slangJson:
    contoh[i] = slangJson[contoh[i]]

print(contoh)

In [ ]:
gkounter = 0 # menggunakan variabel ini untuk mencatat berapa banyak data
             # yang sudah diproses oleh fungsi-fungsi dibawah ini
            
def cleaningText(text):
    global gkounter
    text = re.sub(r'@[A-Za-z0-9]+', '', str(text)) # hapus mentions
    text = re.sub(r'#[A-Za-z0-9]+', '', str(text)) # hapus hashtag
    #Removes unicode strings like "\u002c" and "x96" 
    text = re.sub(r'(\\u[0-9A-Fa-f]+)',r'', str(text))       
    text = re.sub(r'[^\x00-\x7f]',r'',str(text))
    text = re.sub(r'RT[\s]', '',  str(text)) # hapus RT
    text = re.sub(r'(http\S+)|(www\.[^\s]+)', '',  str(text))# hapus link
    #Remove any @Username
    text = re.sub('@[^\s]+','',str(text))
    #Remove additional white spaces
    text = re.sub('[\s]+', ' ', str(text))
    text = re.sub('[\n]+', ' ', str(text))
    text = re.sub(r'[0-9]+', '', str(text)) # hapus angka
    text = re.sub(r'[^0-9A-Za-z \t]|(\w+:\/\/\S+)+','', str (text)) # hapus emoticon
    text = text.replace('\n', ' ')# ganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation)) # hapus semua tanda baca
    text = text.strip(' ') # hapus spasi kosong yang ada di awal dan di akhir teks
    print(f"cleaned {gkounter} text")
    gkounter += 1
    return text

In [ ]:
tweet_sample= "How :) to take control of your #debt https://personal.vanguard.com/us/insights/saving-investing/debt-management.#Best advice for #family #financial #success (@PrepareToWin)"
gkounter = 0
tweet_sample = cleaningText(tweet_sample)
tweet_sample


In [ ]:
gkounter = 0
tweets['text_clean'] = tweets['Text'].apply(cleaningText)
tweets['text_clean'].head()

In [ ]:
contoh1 = '@abcd saya suka masak'
contoh1 = re.sub(r'@[A-Za-z0-9]+', '', str(contoh1)) # hapus mentions
contoh1

In [ ]:
gkounter = 0 # menggunakan variabel ini untuk mencatat berapa banyak data
             # yang sudah diproses oleh fungsi-fungsi dibawah ini
            
def cleaningText(text):
    global gkounter
    text = re.sub(r'@[A-Za-z0-9]+', '', str(text)) # hapus mentions
    text = re.sub(r'#[A-Za-z0-9]+', '', str(text)) # hapus hashtag
    text = re.sub(r'RT[\s]', '',  str(text)) # hapus RT
    text = re.sub(r"http\S+", '',  str(text))# hapus link
    text = re.sub(r'[0-9]+', '', str(text)) # hapus angka
    text = re.sub(r'[^0-9A-Za-z \t]|(\w+:\/\/\S+)+','', str (text)) # hapus emoticon
    text = text.replace('\n', ' ')# ganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation)) # hapus semua tanda baca
    text = text.strip(' ') # hapus spasi kosong yang ada di awal dan di akhir teks
    print(f"cleaned {gkounter} text")
    gkounter += 1
    return text

#casefolding
def casefoldingText(text): # mengubah huruf menjadi huruf kecil
    global gkounter
    text = text.lower() 
    print(f"casefolded {gkounter} text")
    gkounter += 1
    return text

#tokenizing
def tokenizingText(text): # memisahkan string menjadi kumpulan token
    global gkounter
    text = word_tokenize(text) 
    print(f"tokenized {gkounter} text")
    gkounter += 1
    return text

#normalisasi slang words 
def normalizeSlangWords(text):  #normalisasi kata alay/slang menjadi kata baku bahasa indonesia
    global gkounter
    for i in range(len(text)):
      if text[i] in slangJson:
        text[i] = slangJson[text[i]]
    print(f"normalized slang words {gkounter} text")
    gkounter += 1
    return text

#normalisasi slang word - filtering text
#https://github.com/masdevid/ID-Stopwords 
def filteringText(text): # hapus stop words
    global gkounter
    listStopwords = set(stopwords.words('indonesian'))
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered 
    print(f"filtered {gkounter} text")
    gkounter += 1
    return text

#stemming
def stemmingText(text): # buat semua kata menjadi kata baku (contoh: 'melelahkan' => 'lelah')
    global gkounter
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    text = [stemmer.stem(word) for word in text]
    print(f"stemmed {gkounter} text")
    gkounter += 1
    return text

def toSentence(list_words): # fungsi untuk menyambung kembali token-token kata menjadi satu kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence
